## Reading, exploring data

In [3]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
import urllib.request

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filename = 'input.txt'

# download file
urllib.request.urlretrieve(url, filename)

('input.txt', <http.client.HTTPMessage at 0x254d23eaff0>)

In [4]:
# read it in to inspect it
with open(filename, 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print('length of dataset in characters : ', len(text))

length of dataset in characters :  1115394


In [6]:
# let's look at the first 1000 charaacters
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [7]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('vocab size :', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size : 65


## Tokenization, train / val split

In [8]:
#create a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder : take a list of intergers, output a string

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
# let's now encode the entire text dataset and store it into a torch.tensor
import torch # we use PyTorch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100]) # the 1000 characters we looked at earlier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
# Let's now split up the data into trainn and validation sets
n = int(0.9 * len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:] 

In [11]:
# chunk = block
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size] # inputs to the transformer
y = train_data[1:block_size+1]  
for t in range(block_size):
    context = x[:t+1]   
    target = y[t]
    print(f'when input is {context} the target : ', {target})

when input is tensor([18]) the target :  {tensor(47)}
when input is tensor([18, 47]) the target :  {tensor(56)}
when input is tensor([18, 47, 56]) the target :  {tensor(57)}
when input is tensor([18, 47, 56, 57]) the target :  {tensor(58)}
when input is tensor([18, 47, 56, 57, 58]) the target :  {tensor(1)}
when input is tensor([18, 47, 56, 57, 58,  1]) the target :  {tensor(15)}
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target :  {tensor(47)}
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target :  {tensor(58)}


In [13]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    # data는 여기서 불러온다.
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # (high, size)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    # return (batch_size) x (block_size) tensor = 4 * 8 
    return x, y

xb, yb = get_batch('train')
print('input:')
print(xb.shape)
print(xb)
print('target:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target : {target}')


input:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target : 43
when input is [24, 43] the target : 58
when input is [24, 43, 58] the target : 5
when input is [24, 43, 58, 5] the target : 57
when input is [24, 43, 58, 5, 57] the target : 1
when input is [24, 43, 58, 5, 57, 1] the target : 46
when input is [24, 43, 58, 5, 57, 1, 46] the target : 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target : 39
when input is [44] the target : 53
when input is [44, 53] the target : 56
when input is [44, 53, 56] the target : 1
when input is [44, 53, 56, 1] the target : 58
when input is [44, 53, 56, 1, 58] the target : 46
when input

In [14]:
print(xb) # our input to the transformer 

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


### simplest baseline : bigram language model, loss, geeration

In [15]:
import torch
import torch.nn as nn 
from torch.nn import functional as F

torch.manual_seed(1337)  

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) 
        # B : batch size
        # T : Time -> block size
        # C : channel, at this situations, vocab_size 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C) # 2 dimensional
            targets = targets.view(B*T) # 1 dimensional
            loss = F.cross_entropy(logits, targets) # negative log likelihood
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            # self(idx)는 뭘까?
            # nn.Module을 상속받았기에, 자동으로 __call__ 메서드를 갖으며,
            # forward 메서드로 연결된다.
            # 즉, self(idx) == self.forward(idx)
            logits, loss = self(idx)
            # focus only on the last time step, 
            # we pluck out last element at the time dimension
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities (vocab들의 분포 확률을 구한다.)
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution (distribution에서 뽑는다.)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence (뽑은 결과를 idx에 새로 추가한다.)
            idx = torch.cat((idx,idx_next), dim=1)  # (B, T+1)
        return idx
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb,yb)
print(logits.shape)    
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(idx = torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [16]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [17]:
batch_size = 32
for steps in range(1000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.7218432426452637


In [18]:
print(decode(model.generate(idx = torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))


olylvLLko'TMyatyIoconxad.?-tNSqYPsx&bF.oiR;BD$dZBMZv'K f bRSmIKptRPly:AUC&$zLK,qUEy&Ay;ZxjKVhmrdagC-bTop-QJe.H?x
JGF&pwst-P sti.hlEsu;w:w a BG:tLhMk,epdhlay'sVzLq--ERwXUzDnq-bn czXxxI&V&Pynnl,s,Ioto!uvixwC-IJXElrgm C-.bcoCPJ
IMphsevhO AL!-K:AIkpre,
rPHEJUzV;P?uN3b?ohoRiBUENoV3B&jumNL;Aik,
xf -IEKROn JSyYWW?n 'ay;:weO'AqVzPyoiBL? seAX3Dot,iy.xyIcf r!!ul-Koi:x pZrAQly'v'a;vEzN
BwowKo'MBqF$PPFb
CjYX3beT,lZ qdda!wfgmJP
DUfNXmnQU mvcv?nlnQF$JUAAywNocd  bGSPyAlprNeQnq-GRSVUP.Ja!IBoDqfI&xJM AXEHV&DKvRS


### The mathematical trick in self-attention

In [19]:
# consider the following toy example:
import torch
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [20]:
# version 1
# We want x[b, t] = mean_{i <=t} x[b,i]
# 위 뜻은, 정보 전달이(여기선 단순히 평균으로 전달) 현재 토큰 기준으로, 오직 이전 순서의 토큰과 이루어져야한다는 것이다.

xbow =  torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, dim = 0)

In [21]:
# version 2
# more efficient way : using matrix multiply as weighted aggregation
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x # (1,T,T) @ (B,T,C) ---> (B,T,C)

# broad casting의 원칙 1 : 비어있는 차원을 1로 추가한다.
# ex) (T,T) -> (1,T,T)
# broad casting의 원칙 2 : 두 차원의 연산중에 있어서, 적어도 차원 하나는 값이 같아야 한다.
torch.allclose(xbow, xbow2)
# false가 나온다면 부동소숫점 연산 때문일듯.

False

In [22]:
# version 3 : use Softmax
from torch.nn import functional as F
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
 # `dim = -1` means last dimension, i.e., dimension 1 for a (T,T) tensor.

In [23]:
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

$Softmax(x_i) =  \frac{exp(x_i)}{\sum_j{exp(x_j)}}$ 에서,
지수함수는 x가 음의 무한대일때, y가 0에 수렴하므로,
-inf가 입력됐을때의 결과값은 0이다.

#### matrix multiply as weighted aggregation
행렬 곱셈으로 평균을 구하는 trick

In [24]:
torch.tril(torch.ones(3,3)) # .tril : Returns the lower triangular part of the matrix

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [25]:

torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, dim = 1, keepdim = True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a = ')
print(a)
print('--')
print('b = ')
print(b)
print('--')
print('c = ')
print(c)

a = 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b = 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c = 
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [26]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
    # linear trasformation
    # nn.Linear의 W(weight tensor) shape : (out_features, in_features)
    # 실제 연산(linear transformation)은 x @ W^T
    # 한 벡터 공간의 점을 다른 벡터 공간의 점으로 옮기는 mapping과 같다.
key = nn.Linear(in_features = C, out_features = head_size, bias=False)
query = nn.Linear(in_features = C, out_features = head_size, bias=False)
value = nn.Linear(in_features = C, out_features = head_size, bias=False)
k = key(x) # (B, T, head_size=16)
q = query(x) # (B, T, head_size=16)

# communication comes now
wei = q @ k.transpose(-2, -1) # traspose last two dimension
                              # In this case -2 : T,
                              #              -1 : head_size
# (B, T, head_size) @ (B, head_size, T) ---> (B, T, T)
# output is raw affinities between all the nodes
# but we don't want after token's affinities so we do masking.
# for example, 5th token doesn't want to get a 6th, 7th... tokens information.
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))

# exponentiate and normalize
# wei is telling us now in data dependent manner, how much of information to aggregate from any of these tokens in the past.
wei = F.softmax(wei, dim=-1)

v = value(x) # (B, T, head_size=16)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [27]:
wei[0].detach().numpy()

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.15737972, 0.8426203 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.20881522, 0.16458629, 0.6265984 , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.5792409 , 0.11871208, 0.18892112, 0.11312591, 0.        ,
        0.        , 0.        , 0.        ],
       [0.02937923, 0.10517433, 0.0469426 , 0.02756283, 0.790941  ,
        0.        , 0.        , 0.        ],
       [0.017573  , 0.26886913, 0.02150743, 0.00889891, 0.6812242 ,
        0.00192731, 0.        , 0.        ],
       [0.1690685 , 0.40663928, 0.04384262, 0.04155193, 0.10477211,
        0.20119756, 0.0329279 , 0.        ],
       [0.02095483, 0.08428346, 0.05548807, 0.22966747, 0.05728786,
        0.0708937 , 0.2422823 , 0.23914227]], dtype=float32)

- 어텐션은 **커뮤니케이션 메커니즘**이다. 이는 방향성 그래프의 노드들이 서로를 바라보며, 자신을 가리키는 모든 노드로부터 데이터에 따라 달라지는 가중치 합(weighted sum)으로 정보를 취합하는 것으로 볼 수 있다.

- 어텐션에는 공간 개념이 없다. 어텐션은 단순히 벡터들의 집합에 대해 작동한다. 이것이 바로 토큰에 위치 정보를 인코딩(positional encoding)해야 하는 이유이다.

-  물론, 배치 차원(batch dimension)의 각 데이터(example)는 완전히 독립적으로 처리되며 서로 정보를 교환하지 않는다.

- 인코더(encoder) 어텐션 블록에서는 tril을 사용한 마스킹(masking) 라인을 삭제하여 모든 토큰이 서로 소통할 수 있게 한다. 반면, 여기서 설명하는 블록은 삼각 마스킹(triangular masking)을 사용하기 때문에 디코더(decoder) 어텐션 블록이라고 불리며, 주로 언어 모델링과 같은 자기회귀(autoregressive) 설정에서 사용된다.

- **셀프 어텐션**(self-attention)은 키(key)와 밸류(value)가 쿼리(query)와 동일한 소스에서 생성되는 것을 의미한다.
- **크로스 어텐션**(cross-attention)에서는 쿼리는 여전히 x에서 생성되지만, 키와 밸류는 다른 외부 소스(예: 인코더 모듈)로부터 제공된다.

- 스케일드(Scaled) 어텐션은 추가적으로 wei 값을 1/sqrt(head_size)로 나눈다. 이를 통해 입력값인 Q(쿼리), K(키)가 단위 분산(unit variance)일 때, wei 값도 단위 분산을 갖게 되어 소프트맥스(Softmax) 함수가 한쪽으로 치우쳐 너무 포화되지 않고 분산된 상태를 유지하게 된다.

In [28]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5

In [29]:
k.var()

tensor(1.0449)

In [30]:
q.var()

tensor(1.0700)

In [31]:
wei.var()

tensor(1.0918)

In [32]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim = -1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [33]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim = -1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [39]:
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        # calculate the forward pass
        xmean = x.mean(1, keepdim=True) # batch mean
        xvar = x.var(1, keepdim=True, unbiased=True) # batch variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

    def to(self, device):
        self.gamma = self.gamma.to(device)
        self.beta = self.beta.to(device)

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100)
x = module(x)
x.shape

torch.Size([32, 100])

In [40]:
x[:,0].mean(), x[:,0].std() # mean, std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [41]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))